# Projeto 2 - Ciência dos Dados

**Nome: Breno Marti**

**Nome: João Pedro Chacon Ruiz**

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [207]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emojis

In [208]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [209]:
'''
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])
'''

"\n#Dados de autenticação do twitter:\n\n#Coloque aqui o identificador da conta no twitter: @fulano\n\n#leitura do arquivo no formato JSON\nwith open('auth.pass') as fp:    \n    data = json.load(fp)\n\n#Configurando a biblioteca. Não modificar\nauth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])\nauth.set_access_token(data['access_token'], data['access_token_secret'])\n"

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [210]:
#Produto escolhido:
produto = 'Bugatti'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [211]:
'''
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if (not msg.retweeted) and ('RT' not in msg.full_text):
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs'
'''

'\n#Cria um objeto para a captura\napi = tweepy.API(auth)\n\n#Inicia a captura, para mais detalhes: ver a documentação do tweepy\ni = 1\nmsgs = []\nfor msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():\n    if (not msg.retweeted) and (\'RT\' not in msg.full_text):\n        msgs.append(msg.full_text.lower())\n        i += 1\n    if i > n:\n        break\n\n#Embaralhando as mensagens para reduzir um possível viés\nshuffle(msgs\'\n'

Salvando os dados em uma planilha Excel:

In [212]:
'''
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()
'''

"\n#Verifica se o arquivo não existe para não substituir um conjunto pronto\nif not os.path.isfile('./{0}.xlsx'.format(produto)):\n    \n    #Abre o arquivo para escrita\n    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))\n\n    #divide o conjunto de mensagens em duas planilhas\n    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})\n    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)\n\n    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})\n    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)\n\n    #fecha o arquivo\n    writer.save()\n"

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [213]:
raw = pd.read_excel('Bugatti_treino.xlsx')
relevante_treino_raw = raw[raw['Relevância'] == 1]
nao_relevante_treino_raw = raw[raw["Relevância"] == 0]

#### Funções de limpeza de texto:


In [214]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[#@,!\()-.€":?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed   

In [215]:
import emojis

In [216]:
def transform_emojis(text):
    clean = emojis.decode(text)
    return clean

In [217]:
l = ['\n','https','//t','co/']

#### Limpando e transformando os dataframes em séries para a análise:

In [218]:
list1 = []
for tweet in relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list1.append(cleanup(texto))
    
relevante_treino = pd.Series(list1)

Nota do professor: explicar como o código abaixo funciona.

In [219]:
tweets_relevante_treino = relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()

In [220]:
list2 = []
for tweet in nao_relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list2.append(cleanup(texto))
    
nao_relevante_treino = pd.Series(list2)

In [221]:
tweets_nao_relevante_treino = nao_relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()

In [222]:
list3 = []
for tweet in raw['Treinamento']:
    texto = transform_emojis(tweet)
    list3.append(cleanup(texto))
    
tudo = pd.Series(list3)

In [223]:
tweets_treino = tudo.str.replace('|'.join(l),'',regex=True).str.strip()

#### Contando o número de aparições de cada palavra :

In [224]:
list4 = []
for tweet in tweets_treino:
    list4 = list4 + tweet.split()

palavras_universo_treino = pd.Series(list4)

In [225]:
list5 = []
for tweet in tweets_relevante_treino:
    list5 = list5 + tweet.split()

palavras_relevantes_treino = pd.Series(list5)

In [226]:
list6 = []
for tweet in tweets_nao_relevante_treino:
    list6 = list6 + tweet.split()

palavras_nao_relevantes_treino = pd.Series(list6)

#### Frequência relativa de cada palavra:

In [227]:
palavras_universo_treino.value_counts(True)

bugatti         0.044511
de              0.025549
a               0.024750
o               0.023752
que             0.014970
um              0.014770
e               0.013772
do              0.013373
chiron          0.012974
não             0.012375
na              0.010978
carro           0.009182
km/h            0.009182
nome            0.007984
490             0.007984
eu              0.007784
é               0.007784
com             0.007385
em              0.006986
primeira        0.006587
mais            0.006587
seu             0.005988
gente           0.005190
letra           0.005190
para            0.005190
por             0.004990
velocidade      0.004790
da              0.004790
recorde         0.004591
bananaobjeto    0.004591
                  ...   
parado          0.000200
montadora       0.000200
cv0ohmpumh      0.000200
vbxefpqkkh      0.000200
more            0.000200
velocity        0.000200
sugado          0.000200
duda_bri        0.000200
percebi         0.000200


In [228]:
palavras_relevantes_treino.value_counts(True)

bugatti           0.051213
o                 0.037062
a                 0.035714
de                0.035040
chiron            0.030323
km/h              0.025606
que               0.020889
490               0.020889
carro             0.018194
um                0.017520
do                0.015499
em                0.014825
velocidade        0.013477
mais              0.013477
recorde           0.012803
e                 0.011456
é                 0.010108
com               0.010108
não               0.009434
da                0.008760
300               0.007412
mph               0.006739
sport             0.006739
os                0.006065
super             0.005391
chega             0.005391
mundial           0.005391
para              0.004717
por               0.004717
eu                0.004717
                    ...   
pergunto          0.000674
coro              0.000674
alguém            0.000674
conhecidos        0.000674
título            0.000674
percam            0.000674
h

In [229]:
palavras_nao_relevantes_treino.value_counts(True)

bugatti         0.041690
de              0.021554
a               0.020136
o               0.018151
e               0.014748
na              0.013897
não             0.013613
um              0.013613
do              0.012479
que             0.012479
nome            0.011061
primeira        0.009075
eu              0.009075
seu             0.008225
letra           0.007374
gente           0.007090
é               0.006807
bananaobjeto    0.006523
vamos           0.006239
vale            0.006239
jogar           0.006239
com             0.006239
olhar           0.006239
respondanome    0.005956
stop            0.005956
brancofruta     0.005956
bugatticor      0.005672
chiron          0.005672
para            0.005389
carro           0.005389
                  ...   
amo             0.000284
pensar          0.000284
quatrocentos    0.000284
itupeva         0.000284
mateca          0.000284
fxnqayanji      0.000284
bolsa           0.000284
escolham        0.000284
scglfo0s1p      0.000284


In [230]:
P_relevante = palavras_relevantes.size/universo.size
P_relevante

0.2962075848303393

In [231]:
P_nao_relevante = palavras_nao_relevantes.size/universo.size
P_nao_relevante

0.7037924151696606

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [232]:
raw2 = pd.read_excel('Bugatti_teste.xlsx')
relevante_teste_raw2 = raw2[raw2['Relevância'] == 1]
nao_relevante_teste_raw2 = raw2[raw2["Relevância"] == 0]

In [233]:
lista1 = []
for tweet in relevante_teste_raw2['Teste']:
    texto = transform_emojis(tweet)
    lista1.append(cleanup(texto))
    
tweets_relevante_teste = pd.Series(lista1)
tweets_relevante_teste = tweets_relevante_teste.str.replace('|'.join(l),'',regex=True).str.strip()

In [234]:
lista2 = []
for tweet in nao_relevante_teste_raw2['Teste']:
    texto = transform_emojis(tweet)
    lista2.append(cleanup(texto))
    
tweets_nao_relevante_teste = pd.Series(lista2)
tweets_nao_relevante_teste = tweets_nao_relevante_teste.str.replace('|'.join(l),'',regex=True).str.strip()

In [235]:
lista3 = []
for tweet in raw2['Teste']:
    texto = transform_emojis(tweet)
    lista3.append(cleanup(texto))
    
tweets_universo_teste = pd.Series(lista3)
tweets_universo_teste = tweets_universo_teste.str.replace('|'.join(l),'',regex=True).str.strip()

In [236]:
lista4 = []
for tweet in tweets_universo_teste:
    lista4 = lista4 + tweet.split()

palavras_universo_teste = pd.Series(lista4)
palavras_universo_teste.value_counts()

bugatti           137
a                  92
de                 73
o                  66
e                  63
do                 51
que                51
chiron             47
não                43
km/h               42
um                 39
é                  37
490                32
com                31
carro              31
recorde            28
na                 26
uma                23
nome               21
eu                 21
em                 20
mais               19
primeira           18
seu                18
pra                17
300                17
bananaobjeto       17
no                 17
vamos              16
bugatticor         16
                 ... 
jockey              1
publico             1
mangoldmikel        1
franquia            1
revolucionou        1
bolocarro           1
bananacarro         1
capítulos           1
influenciou         1
deu_tigre2°         1
tweet               1
11384e              1
lembrando           1
lego                1
mule      

In [237]:
lista5 = []
for tweet in tweets_relevante_teste:
    lista5 = lista5 + tweet.split()

palavras_relevantes_teste = pd.Series(lista5)
freq_rel_palavras_relevantes_teste = palavras_relevantes_teste.value_counts(True)

In [238]:
freq_rel_palavras_relevantes_teste

bugatti            0.055959
a                  0.039971
chiron             0.030523
o                  0.028343
km/h               0.027616
de                 0.023983
490                0.023256
e                  0.021076
recorde            0.020349
que                0.015262
do                 0.013808
um                 0.013808
carro              0.013081
é                  0.012355
velocidade         0.011628
300                0.010901
em                 0.010174
chega              0.010174
com                0.010174
quebra             0.007267
mais               0.007267
não                0.007267
astonished         0.007267
novo               0.006541
super              0.006541
mundial            0.006541
mph                0.006541
aos                0.005814
vídeo              0.005814
via                0.005814
                     ...   
anuncieseucarro    0.000727
constantemente     0.000727
uniu               0.000727
éeste              0.000727
trazer             0

In [239]:
lista6 = []
for tweet in tweets_nao_relevante_teste:
    lista6 = lista6 + tweet.split()

palavras_nao_relevantes_teste = pd.Series(lista6)
freq_rel_palavras_nao_relevantes_teste = palavras_nao_relevantes_teste.value_counts(True)

In [240]:
freq_rel_palavras_nao_relevantes_teste

bugatti                                                      0.029412
de                                                           0.019608
a                                                            0.018137
e                                                            0.016667
não                                                          0.016176
do                                                           0.015686
que                                                          0.014706
o                                                            0.013235
na                                                           0.011765
uma                                                          0.009804
um                                                           0.009804
nome                                                         0.009804
é                                                            0.009804
eu                                                           0.008333
primeira            

### Montando o Classificardor Naive Bayes

#### Probabilidades de ser relevante ou não relevante

In [241]:
num_palavras_teste = palavras_relevantes_teste.size + palavras_nao_relevantes_teste.size
num_palavras_teste

3416

In [242]:
#probabilidade do tweet ser relevante
prob_relevante_teste = tweets_relevante_teste.size / tweets_universo_teste.size
prob_relevante_teste

0.38

In [243]:
#probabilidade do tweet não ser relevante
prob_nao_relevante_teste = tweets_nao_relevante_teste.size / tweets_universo_teste.size
prob_nao_relevante_teste

0.62

### Como o classificador funcionará:

Todos os tweets presentes na planilha de testes serão, individualmente, analisados pelo classificador e catalogados, em um loop. Depois de todo esse processo, os resultados de cada tweet analisado, que podem ser ou "Relevante" ou "Irrelevante", pelo menos neste primeiro momento, serão adicionados a uma lista com o fim de serem comparados com a análise feita à mão.

In [244]:
'''
def Bayes(relevancia,serie1,serie2):
    #Criando o loop para o cálculo da probabilidade de ser relevante:
    i = 0 
    lista_resultados_numericos = []

    #selecionando o tweet
    for tweets in testudo:
        
        #transformando em lista de str
        tweetr = tweets.split()
        
        # Calculando a probabilidade deste tweet ser relevante dado as suas palavras
        log_probabilidade = 0
        for palavra in tweetr:
        
            #Não deixa que palavras não pertencentes à série bugem o código
            if palavra not in serie1:
                log_probabilidade += math.log(1/3416, 10)
            else:
                log_probabilidade += math.log(serie2[palavra] + 1/3416, 10)
        
        #Guardando os resultados e resetando a variável probabilidade    
        lista_resultados_numericos.append(log_probabilidade + math.log(relevancia, 10))
        log_probabilidade = 0
        i += 1

    return lista_resultados_numerico'
'''

"\ndef Bayes(relevancia,serie1,serie2):\n    #Criando o loop para o cálculo da probabilidade de ser relevante:\n    i = 0 \n    lista_resultados_numericos = []\n\n    #selecionando o tweet\n    for tweets in testudo:\n        \n        #transformando em lista de str\n        tweetr = tweets.split()\n        \n        # Calculando a probabilidade deste tweet ser relevante dado as suas palavras\n        log_probabilidade = 0\n        for palavra in tweetr:\n        \n            #Não deixa que palavras não pertencentes à série bugem o código\n            if palavra not in serie1:\n                log_probabilidade += math.log(1/3416, 10)\n            else:\n                log_probabilidade += math.log(serie2[palavra] + 1/3416, 10)\n        \n        #Guardando os resultados e resetando a variável probabilidade    \n        lista_resultados_numericos.append(log_probabilidade + math.log(relevancia, 10))\n        log_probabilidade = 0\n        i += 1\n\n    return lista_resultados_numerico

In [261]:
smooth = 1e-10 #é uma estimativa e deve mudar
vocabulario = 356000

# defini probabilidade de ser relevante:
def relevante(tweet):
    
    tweet_relevante_4 = tweet.split()

    prob = 1
    
    for p in tweet_relevante_4:
        
        if p in freq_rel_palavras_relevantes_teste:
            prob = prob*(freq_rel_palavras_relevantes_teste[p]*palavras_relevantes_teste.size+smooth)/(palavras_relevantes_teste.size+smooth*vocabulario)*1000
        else:
            prob = prob*(smooth)/(palavras_relevantes_teste.size+smooth*vocabulario)*1000
            
    prob = prob * prob_relevante_teste
    prob = math.log10(prob)
    #print(prob)
    return prob

# defini probabilidade de ser irrelevante:
def irrelevante(tweet):
    
    tweet_irrelevante_4 = tweet.split()
            
    prob = 1
    
    for p in tweet_irrelevante_4:
        
        if p in freq_rel_palavras_nao_relevantes_teste:
            prob = prob*(freq_rel_palavras_nao_relevantes_teste[p]*palavras_nao_relevantes_teste.size+smooth)/(palavras_nao_relevantes_teste.size+smooth*vocabulario)*1000
        else:
            prob = prob*(smooth)/(palavras_nao_relevantes_teste.size+smooth*vocabulario)*1000
            
    prob = prob * prob_nao_relevante_teste
    prob = math.log10(prob)
    #print(prob)
    return prob

In [281]:
i=0
lista_resultados = []
for tweet in tweets_universo_teste:
    
    if relevante(tweet) > irrelevante(tweet):
           lista_resultados.append(1)
            
    if relevante(tweet) < irrelevante(tweet):
        lista_resultados.append(0)
        
    i += 1
    
lista_resultados

[0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0]

In [282]:
i = 0
verdadeiros_positivos = 0
falsos_positivos = 0
verdadeiros_negativos = 0
falsos_negativos = 0

for tweet in raw2['Relevância']:
    if tweet == lista_resultados[i] and lista_resultados[i] == 1:
        verdadeiros_positivos += 1
    
    if tweet == lista_resultados[i] and lista_resultados[i] == 0:
        verdadeiros_negativos += 1
    
    if tweet != lista_resultados[i] and lista_resultados[i] == 1:
        falso_positivos += 1
        
    if tweet != lista_resultados[i] and lista_resultados[i] == 0:
        falsos_negativos += 1
        
    i += 1
        
print(verdadeiros_positivos)
print(falsos_positivos)
print(verdadeiros_negativos)
print(falsos_negativos)

76
0
124
0


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**